In [ ]:
%matplotlib inline

In [ ]:
import os
import glob
import json
import datetime
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

from IPython.core.display import HTML

In [ ]:
def objFromFile(filename):
    with open(filename, 'r') as fobj:
        json_str = fobj.readline()
        fobj.close()
    return json.loads(json_str)

In [ ]:
data_home = os.path.expanduser('~/Documents/Tesla')
folders = glob.glob('{}/2*'.format(data_home))
folders.sort()

t = []
d = []
for folder in folders:
    files = glob.glob('{}/*.json'.format(folder))
    files.sort()
    dd = []
    for file in files:
        #print('{}'.format(file))
        data = objFromFile(file)
        dd.append(data)
    d.append(dd)
    o = d[-1][-1]['charge_state']
    t.append(datetime.datetime.fromtimestamp(o['timestamp'] / 1000))
    print('{} -> {}% {} {}'.format(os.path.basename(file), o['battery_level'], o['charging_state'], t[-1].weekday()))

In [ ]:
# Find the previous Sunday
t0 = t[0]
oneday = datetime.timedelta(days=1)
while t0.weekday() != 6:
    t0 -= oneday
# display('{}'.format(t0.strftime('%Y/%m/%d %H:%M %A')))

# The first Sunday
t0 = datetime.datetime.strptime(t0.strftime('%Y-%m-%d'), '%Y-%m-%d')
display('{}'.format(t0.strftime('%Y/%m/%d %H:%M %A')))

# Group the weeks
tt = []
dd = []
k = 0
# while k < len(t):
for i in range(4):
    tw = []
    dw = []
    for i in range(7):
        if k < len(t) and t0 <= t[k] and t[k] < (t0 + oneday):
            tw.append(t[k])
            dw.append(d[k])
            k += 1
        else:
            tw.append(t0)
            dw.append(None)
        t0 += oneday
    tt.append(tw)
    dd.append(dw)
display(tt)

In [ ]:
# view = WeeklyChart()

In [ ]:
figsize = (900, 640)

# 7w + 8o = figsize[0]
# o = 0.05w

o = 0.04
w = figsize[0] / (7 + 8 * o)
h = figsize[1] / (4.5 + 5 * o)

o = w * o

In [ ]:
o

In [ ]:
def compute_xr(x, w):
    return x + 0.96 * w

code = '<html>'
code += '<head>'
code += '<meta charset="UTF-8">'
code += '<style type="text/css">'
code += '@font-face {font-family:"Helvetica Neue"; src:url(fonts/HelveticaNeueLight.ttf)}'
code += 'html {font-family: "Helvetica Neue", Arial, sans-serif; font-size:10pt; }'
code += '.box-container {{display:block; position:relative; left:50%; margin-left:-{}px; width:100%; min-height:{}px; background-color:white;}}'.format(0.5 * figsize[0], figsize[1])
code += '.box {{display:block; position:absolute; width:{}px; height:{}px; border:solid 1px #aaa;}}'.format(w, h)
code += '.title {display:block; position:absolute; top:10px; } '
code += '.titleMonth, .titleYear {display:inline-block; font-size:2.0em;} '
code += '.titleMonth {font-weight:500;} '
code += '.dayOfWeek {{display:block; position:absolute; width:{}px; height:{}px; text-align:right;}}'.format(w, 0.2 * h)
code += '.dayLabel, .titleDayLabel {display:block; position:absolute; top:5px; right:5px; font-size:1.1em; z-index:100} '
code += '.titleDayLabel {font-size:1.2em; } '
code += '.batteryLevel {display:block; position:absolute; bottom:0; width:100%; z-index:0 } '

code += '.info {display:block; position:absolute; top:40%; width:100%; margin:0; padding:0; }'
code += '.batteryNumber {display:block; font-size:1.8em; font-weight:500; font-stretch:extra-expanded; width:100%; text-align:center; text-valign:middle; } '
code += '.timeLabel {display:block; width:100%; margin-top:0.5em; text-align:center; } '
code += '.dataCount {display:block; width:100%; text-align:center; } '

code += '.iconBar {display:block; position:absolute; top:8px; left:8px; width:100%; }'
code += 'img.icon {float:left; margin:0; width:18px; padding:2px; } '
code += '</style>'
code += '</head>'

code += '<body>'

code += '<div class="box-container">'

code += '<div class="title">'
code += '<span class="titleMonth">{}</span>&nbsp; <span class="titleYear">{}</span>'.format(tt[0][0].strftime('%B'), tt[0][0].strftime('%Y'))
code += ' <span class="titleMonth">&nbsp; 🚘 🔋 ⚡️</span>'
code += '</div>'

m = tt[0][0].month

for i in range(7):
    x = i * (w + o) + o
    y = 0.28 * h + o
    code += '<div class="dayOfWeek" style="left:{:.2f}px; top:{:.2f}px">'.format(x, y,)
    code += '<span class="titleDayLabel">{}</span>'.format(tt[0][i].strftime('%a'))
    code += '</div>'
    
m1 = 0.0
for j in range(len(tt)):
    for i in range(len(tt[j])):
        x = i * (w + o) + o
        y = j * (h + o) + o + 0.5 * h
        xc = x + 0.5 * w
        xr = compute_xr(x, w)
        
        # Background
        code += '<div class="box" style="left:{:.2f}px; top:{:.2f}px">'.format(x, y)

        # The day label.
        if m != tt[j][i].month:
            m = tt[j][i].month
            dayString = '{}'.format(tt[j][i].strftime('%b %-d'))
        else:
            dayString = '{}'.format(tt[j][i].strftime('%-d'))
        code += '<span class="dayLabel">{}</span>'.format(dayString)
        
        # Skip the day if there is no data
        if dd[j][i] is None:
            code += '</div>'
            continue

        # Battery level
        dayArray = dd[j][i]
        batteryLevel = dayArray[-1]['charge_state']['battery_level']
        color = '#99ff00'
        if batteryLevel <= 60.0:
            color = '#ffcc00'
        code += '<div class="batteryLevel" style="background-color:{}; height:{}%"></div>'.format(color, batteryLevel)

        # Icon bar
        code += '<div class="iconBar">'
        if any([d['charge_state']['charging_state'] == 'Charging' for d in dayArray]):
            code += '<img class="icon" src="blob/charge-0.png">'
        # Calculate total miles driven
        if m1 == 0.0:
            miles = dayArray[-1]['vehicle_state']['odometer'] - dayArray[0]['vehicle_state']['odometer']
        else:
            miles = dayArray[-1]['vehicle_state']['odometer'] - m1
        m1 = dayArray[-1]['vehicle_state']['odometer']
        if miles > 20:
            code += '<img class="icon" src="blob/wheel.png">'
        code += '</div>'

        # Lines of information
        code += '<div class="info">'
        code += '<span class="batteryNumber">{}%</span>'.format(batteryLevel)
        code += '<span class="timeLabel">({}) - {}</span>'.format(len(dayArray), tt[j][i].strftime('%I:%M %p'))
        code += '<span class="dataCount">{:.1f} mi</span>'.format(miles)
        code += '</div>'
        
        code += '</div>'
        
code += '</div>'
code += '</body>'
code += '</html>'

In [ ]:
HTML(code)

In [ ]:
with open('test.html', 'w') as fid:
    fid.write(code)
    fid.close()

In [ ]:
fig = plt.figure(figsize=(12, 3), dpi=96)
plt.subplots_adjust(bottom=0.2)
plt.xticks(rotation=35)
ax = plt.gca()
xfmt = matplotlib.dates.DateFormatter('%Y-%m-%d')
ax.xaxis.set_major_formatter(xfmt)
# plt.plot(t, b, '.-')
plt.stem(t, b)
plt.grid()

In [ ]:
t = []
b = []
for file in files:
    data = objFromFile(file)
    o = data['drive_state']
    
    o = data['charge_state']
    t.append(datetime.datetime.fromtimestamp(o['timestamp'] / 1000))
    b.append(o['battery_level'])
    print('{} -> {}% {}'.format(os.path.basename(file), o['battery_level'], o['charging_state']))

In [ ]:
fig = plt.figure(figsize=(12, 3), dpi=96)
plt.subplots_adjust(bottom=0.2)
plt.xticks(rotation=25)
ax = plt.gca()
xfmt = matplotlib.dates.DateFormatter('%Y-%m-%d %H:%M:%S')
ax.xaxis.set_major_formatter(xfmt)
# plt.plot(t, b, '.-')
plt.stem(t, b)

In [ ]:
for file in files[-5:]:
    data = objFromFile(file)
    display(file)
    display(data['drive_state'])